<a href="https://colab.research.google.com/github/robinjameslee/Deribit-Solana-Options-Implied-Futures-Yield/blob/main/Deribit_Solana_Options_Implied_Futures_Yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script aims to have a look at the implied futures premium of Solana options listed on Deribit.

We first query the Solana option chain, then use Put-Call parity to price the implied Solana Futures price.

With that we can price the futures premium, and the implied yield if we do a Cash & Carry trade

Data are queried using Deribit's API: [docs.deribit.com](https://docs.deribit.com)

In [47]:
import json
import requests
import pandas as pd
import datetime
import numpy as np

pd.set_option('display.float_format', '{:.10f}'.format)

def get_instruments(currency, kind, expired='false'):
  url = 'https://www.deribit.com/api/v2/public/get_instruments?'
  parameters = {'currency': currency, 'kind': kind, 'expired': expired}
  res = requests.get(url, params=parameters)
  res_dict = json.loads(res.content)['result']
  return res_dict

def get_spot(underlying):
  url = 'https://www.deribit.com/api/v2/public/get_index_price?'
  res = requests.get(url, params={'index_name': underlying})
  spot_price = json.loads(res.content)['result']['index_price']
  return spot_price

def get_book_summary_by_currency(currency, kind):
  url = 'https://www.deribit.com/api/v2/public/get_book_summary_by_currency?'
  parameters = {'currency': currency, 'kind': kind}
  res = requests.get(url, params=parameters)
  res_dict = json.loads(res.content)['result']
  return res_dict

underlying = 'SOL' #SOL or MATIC or XRP
underlying_spot = get_spot(f'{underlying.lower()}_usdc')
underlying_options = get_instruments(currency='USDC', kind='option')
underlying_options_book_summary = pd.DataFrame(get_book_summary_by_currency(currency='USDC', kind='option'))

In [48]:
filtered_option_df = pd.DataFrame([], columns=['instrument_name', 'expiry', 'option_type', 'strike'])
today_plus_20 = datetime.datetime.today() + datetime.timedelta(days=20)

#filter options based on our conditions
for options in underlying_options:
  expiry = pd.to_datetime(options['expiration_timestamp'], unit='ms')
  strike = options['strike']
  option_type = options['option_type']
  if options['base_currency'] == underlying and expiry >= today_plus_20 and strike >= underlying_spot * 0.8 and strike <= underlying_spot * 1.2:
    filtered_option_df.loc[len(filtered_option_df)] = [options['instrument_name'], expiry, option_type, strike]

filtered_option_df.head()

,instrument_name,expiry,option_type,strike
0,SOL_USDC-30AUG24-145-C,2024-08-30 08:00:00,call,145.0000000000
1,SOL_USDC-30AUG24-145-P,2024-08-30 08:00:00,put,145.0000000000
2,SOL_USDC-30AUG24-150-C,2024-08-30 08:00:00,call,150.0000000000
3,SOL_USDC-30AUG24-150-P,2024-08-30 08:00:00,put,150.0000000000
4,SOL_USDC-30AUG24-155-C,2024-08-30 08:00:00,call,155.0000000000


In [49]:
underlying_options_book_summary = underlying_options_book_summary[['instrument_name', 'underlying_price', 'mark_iv', 'mark_price', 'bid_price', 'ask_price']]
df = filtered_option_df.merge(underlying_options_book_summary, on='instrument_name')
df['day_to_expiry'] = (df['expiry'] - datetime.datetime.today()).dt.days
df

,instrument_name,expiry,option_type,strike,underlying_price,mark_iv,mark_price,bid_price,ask_price,day_to_expiry
0,SOL_USDC-30AUG24-145-C,2024-08-30 08:00:00,call,145.0000000000,179.8394000000,81.7700000000,39.8961120000,40.5000000000,NaN,38
1,SOL_USDC-30AUG24-145-P,2024-08-30 08:00:00,put,145.0000000000,179.8394000000,81.7700000000,5.0550610000,4.9000000000,5.2000000000,38
2,SOL_USDC-30AUG24-150-C,2024-08-30 08:00:00,call,150.0000000000,179.8394000000,81.4400000000,36.4489740000,7.3000000000,NaN,38
3,SOL_USDC-30AUG24-150-P,2024-08-30 08:00:00,put,150.0000000000,179.8394000000,81.4400000000,6.3509750000,6.2000000000,6.5000000000,38
4,SOL_USDC-30AUG24-155-C,2024-08-30 08:00:00,call,155.0000000000,179.8394000000,80.8800000000,32.9304360000,NaN,NaN,38
5,SOL_USDC-30AUG24-155-P,2024-08-30 08:00:00,put,155.0000000000,179.8394000000,80.8800000000,7.8100130000,7.6000000000,8.0000000000,38
6,SOL_USDC-30AUG24-160-C,2024-08-30 08:00:00,call,160.0000000000,179.8394000000,80.5100000000,29.6517460000,NaN,NaN,38
7,SOL_USDC-30AUG24-160-P,2024-08-30 08:00:00,put,160.0000000000,179.8394000000,80.5100000000,9.5107470000,9.3000000000,9.7000000000,38
8,SOL_USDC-30AUG24-165-C,2024-08-30 08:00:00,call,165.0000000000,179.8394000000,80.3000000000,26.4863520000,NaN,NaN,38
9,SOL_USDC-30AUG24-165-P,2024-08-30 08:00:00,put,165.0000000000,179.8394000000,80.3000000000,11.4600430000,11.2000000000,11.7000000000,38


In [50]:
#Loop through all strikes, and calculate the implied futures price using the Put-Call Parity
strikes_list = df['strike'].unique()
futures_df = pd.DataFrame([], columns=['expiry', 'day_to_expiry', 'strike', 'call', 'put', 'futures_price', 'futures_premium', 'annualized_futures_premium'])
for strike in strikes_list:
  expiry = df[(df['option_type'] == 'call') & (df['strike'] == strike)]['expiry'].values[0] #randomly get one for now
  day_to_expiry = df[(df['option_type'] == 'call') & (df['strike'] == strike)]['day_to_expiry'].values[0] #randomly get one for now
  call_price = df[(df['option_type'] == 'call') & (df['strike'] == strike)]['bid_price'].values[0]
  put_price = df[(df['option_type'] == 'put') & (df['strike'] == strike)]['ask_price'].values[0]
  futures_price = call_price - put_price + strike #Put-Call Parity: Call - Put = Futures - Strike
  futures_premium = futures_price /  underlying_spot - 1
  annualized_futures_premium = (1 + futures_premium) ** (365 / day_to_expiry) - 1 #Annauzlied yield, (1 + premium %) ** (365 / day to expiration) - 1
  futures_df.loc[len(futures_df)] = [expiry, day_to_expiry, strike, call_price, put_price, futures_price, futures_premium, annualized_futures_premium]

futures_df

,expiry,day_to_expiry,strike,call,put,futures_price,futures_premium,annualized_futures_premium
0,2024-08-30 08:00:00,38,145.0000000000,40.5000000000,5.2000000000,180.3000000000,0.0025661913,0.0249228794
1,2024-08-30 08:00:00,38,150.0000000000,7.3000000000,6.5000000000,150.8000000000,-0.1614698744,-0.8157630656
2,2024-08-30 08:00:00,38,155.0000000000,NaN,8.0000000000,NaN,NaN,NaN
3,2024-08-30 08:00:00,38,160.0000000000,NaN,9.7000000000,NaN,NaN,NaN
4,2024-08-30 08:00:00,38,165.0000000000,NaN,11.7000000000,NaN,NaN,NaN
5,2024-08-30 08:00:00,38,170.0000000000,24.2000000000,13.9000000000,180.3000000000,0.0025661913,0.0249228794
6,2024-08-30 08:00:00,38,175.0000000000,22.9000000000,16.3000000000,181.6000000000,0.0097948993,0.0981476520
7,2024-08-30 08:00:00,38,180.0000000000,20.6000000000,19.0000000000,181.6000000000,0.0097948993,0.0981476520
8,2024-08-30 08:00:00,38,185.0000000000,18.4000000000,21.8000000000,181.6000000000,0.0097948993,0.0981476520
9,2024-08-30 08:00:00,38,190.0000000000,16.5000000000,24.9000000000,181.6000000000,0.0097948993,0.0981476520
